To run this code, you need the followin python packages
* pyspark



In [59]:
import sys
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import col, size, explode

import os
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="python3"

class Tweet:
    hashtag = []
    location = []
    time = ""

In [60]:
# create Spark context with Spark configuration
conf = SparkConf().setAppName("Twitter data processing. Text file to RDD")
sc = SparkContext.getOrCreate(conf=conf)
    
sqlContext = SQLContext(sc)
df = sqlContext.read.json("stream_output_oct24.json")

df2 = df.withColumn("hashtag", explode("entities.hashtags"))
df2.createOrReplaceTempView("tweets")

In [108]:
#df.printSchema()
#df.show(1)

res = sqlContext.sql("SELECT created_at, hashtag, id, coordinates  from tweets where place IS NOT Null OR coordinates IS NOT Null or user.location IS NOT Null")
#rel.select("created_at", "place", "entities.hashtags").show(2)


rdd = res.rdd.map(lambda x: (x.hashtag.text, x))
#print(rdd.take(1))
groupRDD = rdd.groupByKey().map(lambda x : (x[0], list(x[1])))
#print(rdd.take(10))
pairs = groupRDD.filter(lambda x: len(x[1]) > 1)



In [107]:
pair2 = groupRDD.map(lambda x: (x[0], len(x[1]) )).sortBy(lambda s: s[1], 0)
print(pair2.take(5))

[('StandWithHongKong', 1299), ('HongKong', 1211), ('StandUpForCatalonia', 382), ('Chinese', 339), ('StandWithHK', 335)]
